In [ ]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

**Notes:** 

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

The notebook exploits [pyVO](https://pyvo.readthedocs.io/en/latest/), an advanced library  of The [Virtual Observatory](https://ivoa.net/).

[Astroquery](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html) (not used here) is a well-documented, user-friendly alternative.

--------------------------------------------------------

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1
**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [ ]:
# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/390/466"

We first retrieve the catalogue information.

In [ ]:
# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/390/466"
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [ ]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [ ]:
voresource.creators[0]

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [ ]:
tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")

In [ ]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

The actual data can then be accessed using any of the ``access_modes`` of the voresource.

In [ ]:
voresource.access_modes()

The web access is found by following the ``reference_url``

In [ ]:
voresource.reference_url

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [ ]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

In [ ]:
tap_records.table.columns

In [ ]:
tables_names

<!-- section position -->

We will explore the other access modes here.

### 3.2. Execute a cone search query

We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) protocol of the virtual observatory (only available for tables with positions).

Finding the conesearch service that you want to use (there is usually one per table):

Let's use the first one for this example. 

In [ ]:
# we get the conesearch  service associated to the first table
conesearch_interface = voresource.get_interface(service_type='conesearch', 
                                                keyword='J/MNRAS/390/466/galaxies',
                                                lax=True)
conesearch_service = conesearch_interface.to_service()

We adapt the radius and position of the center to our needs:

In [ ]:
conesearch_radius = 1 / 60.0  # in degrees
conesearch_center = (27.817383, 22.366755)

In [ ]:
conesearch_records = conesearch_service.search(
    pos=conesearch_center,
    sr=conesearch_radius,
)
conesearch_records

## 4. Get the catalogue coverage

VizieR also provides the coverage of each catalogue with Multiple Order Coverage (<a href='https://ivoa.net/documents/MOC/'>MOC</a>) at order 10 -- meaning with a 0.001 rad spacial resolution.

In [ ]:
# retrieve the MOC
catalogue_coverage = MOC.from_vizier_table(CATALOGUE)
catalogue_coverage.display_preview()

The quick preview is the whole sky in equatorial coordinates. Red represent zones where the catalog has data, black areas are empty zones.

We can also plot the coverage with ``matplotlib``.

In [ ]:
fig = plt.figure(figsize=(5, 5))
wcs = catalogue_coverage.wcs(fig)
ax = fig.add_subplot(projection=wcs)
catalogue_coverage.fill(ax=ax, wcs=wcs, alpha=0.5, color="blue")

or with [ipyaladin](https://github.com/cds-astro/ipyaladin)

In [ ]:
aladin = Aladin()
aladin

We switch the target to the center of the conesearch

In [ ]:
aladin.target = "27.817383 +22.366755"

We can add the tables and coverage in this ipyaladin widget:

In [ ]:
aladin.add_table(conesearch_records.to_table(), color="lightskyblue", shape="plus")

If you don't see the points from the conesearch result, try zooming in the widget with your mouse wheel.

We can also add the result of the TAP query:

In [ ]:
aladin.add_table(tap_records.to_table())

In [ ]:
aladin.add_moc(
    catalogue_coverage,
    fill=True, opacity=0.5, color="pink", edge=True
)

For more information about manipulating multi-order coverages, see [MOCpy documentation](https://cds-astro.github.io/mocpy/), and about using the `ipyaladin` widget, see [its documentation](https://cds-astro.github.io/ipyaladin/).

In [ ]:
import pyvo as vo

# Set up the TAP service
service = vo.dal.TAPService("http://tapvizier.cds.unistra.fr/TAPVizieR/tap")


# Define the query to extract mass (or luminosity) and maximum velocity
query = """
SELECT TOP 10 *
FROM "J/MNRAS/390/466/galaxies"
"""
# Execute the query
result = service.search(query)

# Convert the result to a pandas DataFrame
galaxy_data_10 = result.to_table().to_pandas()
galaxy_data_10.columns

In [ ]:
import pyvo as vo
import numpy as np

R0=14.01E9 # 4D radius of the Universe in lyr
# Set up the TAP service
service = vo.dal.TAPService("http://tapvizier.cds.unistra.fr/TAPVizieR/tap")


# Define the query to extract mass (or luminosity) and maximum velocity
query = """
SELECT
    Name, 
    BMAG,  -- Absolute B magnitude, can be converted to mass or used as a luminosity proxy
    Vmax,   -- Maximum rotation velocity km/s
    Dist   -- Distance Mpc
FROM
    "J/MNRAS/390/466/galaxies"
WHERE
    BMAG IS NOT NULL AND
    Vmax IS NOT NULL
"""

# Execute the query
result = service.search(query)

# Convert the result to a pandas DataFrame
galaxy_data = result.to_table().to_pandas()

# Constants
M_sun = 5.48  # Solar absolute magnitude in the B-band
ML_ratio = 5  # Example M/L ratio, adjust based on your data or literature

# Calculate luminosity relative to the Sun
galaxy_data["Dist_lyr"]= 3_261_563* galaxy_data["Dist"]/R0
galaxy_data['z'] = galaxy_data.Dist_lyr/(1-galaxy_data.Dist_lyr)
galaxy_data['Luminosity'] = 10**(-0.4 * (galaxy_data['BMAG'] - M_sun))

# Estimate mass
galaxy_data['Mass'] = galaxy_data['Luminosity'] * ML_ratio

# Check the first few entries
print(galaxy_data[['Name', 'BMAG', 'Luminosity', 'Mass', 'Vmax', 'Dist_lyr', 'z' ]].head())



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'galaxy_data' is the DataFrame with your data
galaxy_data['BMAG'] = galaxy_data['BMAG']  # Using -log10 because brighter galaxies have smaller magnitude values
galaxy_data['LogVmax'] = np.log10(galaxy_data['Vmax'])

plt.figure(figsize=(10, 6))
plt.scatter(galaxy_data['LogVmax'], galaxy_data['BMAG'], c='blue', edgecolor='black')
plt.title('B-band Magnitude vs. log10 of Maximum Velocity')
plt.xlabel('log10(Vmax [km/s])')
plt.ylabel('BMAG')
plt.grid(True)
plt.show()


To derive a proxy for mass using distance, you'll need to adjust the luminosity values you've calculated from the B-band magnitudes. The luminosity \( L \) in terms of the absolute magnitude \( M \) can be expressed by the formula:

$ L = 10^{-0.4(M - M_{\text{sun}})} $

where $ M_{\text{sun}} $ is the absolute magnitude of the Sun in the B-band (about 5.48).

However, to calculate luminosity directly from the apparent magnitude and distance, you can use the distance modulus:

$ M = m - 5 \log_{10}(d) + 5 $

where:
-  M is the absolute magnitude,
-  m is the apparent magnitude,
-  d is the distance in parsecs.

If you already have the distance or can calculate it from the redshift (using a cosmological model or approximation if the redshift \( z \) values are available), you can calculate the absolute magnitude and then the luminosity.

Do you have the distances available, or would you need help deriving them from redshift or another method? If you have distances or redshifts in any of your data files, please let me know how you'd like to proceed!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'galaxy_data' is the DataFrame with your data
galaxy_data['LogMass'] = np.log10( galaxy_data['Mass'])  # Using -log10 because brighter galaxies have smaller magnitude values
galaxy_data['LogVmax'] = np.log10(galaxy_data['Vmax'])

plt.figure(figsize=(10, 6))
plt.scatter(galaxy_data['LogVmax'], galaxy_data['LogMass'], c='blue', edgecolor='black')
plt.title('Log10 Mass vs. log10 of Maximum Velocity')
plt.xlabel('log10(Vmax [km/s])')
plt.ylabel('log10 Mass')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generating some example data for demonstration
np.random.seed(0)
vmax = galaxy_data["LogVmax"]
mass = galaxy_data["LogMass"]

# Performing a linear regression to find the Tully-Fisher Coefficient
coefficients = np.polyfit(vmax, mass, 1)
tully_fisher_coefficient = coefficients[0]

# Using the fitted line to calculate predicted values
predicted_mass = np.polyval(coefficients, vmax)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(vmax, mass, c='blue', edgecolor='black', label='Data')
plt.plot(vmax, predicted_mass, 'r-', label=f'Fit Line: slope = {tully_fisher_coefficient:.2f}')
plt.title(f'Log10 Mass vs. log10 of Maximum Velocity\nTully-Fisher Coefficient: {tully_fisher_coefficient:.2f}')
plt.xlabel('log10(Vmax [km/s])')
plt.ylabel('log10 Mass')
plt.text(2.3, 9.1, 'vizier_jmnras390466', fontsize=12, color='red')
plt.grid(True)
plt.legend()
plt.savefig("./Drawing_For_Publications/GHasp_TullyFisher_M_vs_Vmax.png")
plt.show()

tully_fisher_coefficient


In [ ]:
np.sum([ "7831" in galaxy_data.Name])

In [ ]:
galaxy_data.Name


In [ ]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import astropy 

galaxy_names = [x for x in galaxy_data.Name ]

for name in galaxy_names:
    # Get coordinates for the object
    obj_coords = coords.SkyCoord.from_name(name) 
    # print(obj_coords)

    # Use coordinates for cross-identification
    sdss_obj = SDSS.query_crossid(obj_coords) 
    print(name, sdss_obj)
    if sdss_obj is None:
        print(name, " not found")
        continue
    # objid = sdss_obj['objID'][0] 

    # # Construct the SQL query with the objID (same as before)
    # query = f"""SELECT p.objID, p.ra, p.dec, s.z, s.zErr, 
    #               p.modelMag_u, p.modelMag_g, p.modelMag_r, 
    #               p.modelMag_i, p.modelMag_z 
    #               FROM PhotoObj AS p JOIN SpecObj AS s ON s.bestobjid = p.objid 
    #               WHERE p.objid = {objid}""" 

    # result = SDSS.query_sql(query)
    # galaxy_sdss = result.to_pandas()
    # print(galaxy_sdss)

In [ ]:
z = result["z"]
distance = 

In [ ]:
result.columns

In [ ]:
from astroquery.utils.tap.core import TapPlus

# Connect to an appropriate TAP service
url = 'http://tapvizier.u-strasbg.fr/TAPVizieR/tap'
tap_service = TapPlus(url=url)

# Construct a query to retrieve data
query = """
SELECT TOP 10
  p.objID, p.ra, p.dec, s.z, s.zErr, p.modelMag_u, p.modelMag_g, p.modelMag_r, p.modelMag_i, p.modelMag_z
FROM
  sdss_dr16.PhotoObj AS p
JOIN
  sdss_dr16.SpecObj AS s ON s.bestobjid = p.objid
WHERE
  p.objid IN ('UGC 12893', 'UGC 00089', 'UGC 00094', 'UGC 00508', 'UGC 00528')
"""

result = tap_service.search(query)
print(result.to_table())


In [ ]:
from astroquery.sdss import SDSS
import pandas as pd

galaxy_names = [
    'UGC 12893', 'UGC 00089', 'UGC 00094', 'UGC 00508', 'UGC 00528',  # truncated for brevity
]

results = []

for name in galaxy_names:
    query = f"""
    SELECT
        p.objID, p.ra, p.dec, s.z, s.zErr,
        p.modelMag_u, p.modelMag_g, p.modelMag_r, p.modelMag_i, p.modelMag_z
    FROM
        PhotoObj AS p
        JOIN SpecObj AS s ON s.bestobjid = p.objid
    WHERE
        p.objid = '{name.strip()}'
    """
    try:
        result = SDSS.query_sql(query)
        if result is not None:
            results.append(result.to_pandas())
    except Exception as e:
        print(f"Failed to retrieve or parse data for {name}: {str(e)}")

# Combine all results into a single DataFrame
full_results = pd.concat(results, ignore_index=True)

# Display the combined results
print(full_results.head())


In [ ]:
from astroquery.utils.tap.core import TapPlus
vizier_tap_url = "http://tapvizier.u-strasbg.fr/TAPVizieR/tap"
tap_service = TapPlus(url=vizier_tap_url)

# Construct an ADQL query
query = """
SELECT TOP 10
    p.objID, p.ra, p.dec, s.z, s.zErr,
    p.modelMag_u, p.modelMag_g, p.modelMag_r, p.modelMag_i, p.modelMag_z
FROM
    sdss_dr16.PhotoObj AS p
JOIN
    sdss_dr16.SpecObj AS s ON s.bestobjid = p.objid
WHERE
    p.objid IN ('UGC 12893', 'UGC 00089', 'UGC 00094', 'UGC 00508', 'UGC 00528')
"""

# Execute the query using query_adql or query_job
job = tap_service.launch_job_async(query)
result = job.get_results()

# Show the results
print(result)


In [ ]:
from astroquery.utils.tap.core import TapPlus

# Connect to VizieR TAP service
vizier_tap_url = "http://tapvizier.u-strasbg.fr/TAPVizieR/tap"
tap_service = TapPlus(url=vizier_tap_url)

# Construct an ADQL query
query = """
SELECT TOP 10
    *
FROM
    "V/147/sdss12"
WHERE
    1=1
"""

# Execute the query using query_adql or query_job
job = tap_service.launch_job_async(query)
result = job.get_results()

# Show the results
print(result)


In [ ]:
from astroquery.sdss import SDSS

# Define your query
query = """
SELECT TOP 10
p.*
FROM PhotoObj as p
where 
p.objid = 'UGC 12893'
"""

# Execute the query
result = SDSS.query_sql(query)

# Print the results
print(result)

In [ ]:
from astroquery.skyview import SkyView
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import astropy.units as u

# Resolve the object name to coordinates
pos = SkyView.get_image_list(position='UGC 12893', survey=['SDSS'], pixels=0)

# If position resolution is successful, query around that position
if pos:
    coord = coords.SkyCoord.from_name('UGC 12893')
    result = SDSS.query_region(coord, radius=2*u.arcmin, photoobj_fields=['objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z'])

    # Print the results
    print(result)
else:
    print("Failed to resolve the name to coordinates.")


In [ ]:
from astroquery.skyview import SkyView

# List all available surveys
surveys = SkyView.list_surveys()
print("Available Surveys:", surveys)


In [ ]:
from astroquery.skyview import SkyView
from astropy.coordinates import SkyCoord

# Resolve the object name to coordinates
coord = SkyCoord.from_name('UGC 12893')

# Fetch the image
image_paths = SkyView.get_image_list(position=coord, survey=['SDSSdr7g'])

# Download the first image to view it
images = SkyView.get_images(position=coord, survey=['SDSSdr7g'], radius=0.1 * u.deg)

# Printing image paths and display the image if needed
print("Image URLs:", image_paths)
print("Fetched Images:", images)
